# Exploring State of The Art Models for Question Answering. 

Importing the Libraries.

In [1]:
from transformers import pipeline
from transformers import BertTokenizer, TFBertForQuestionAnswering
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMultipleChoice

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Benchmark model.

In [2]:
nlp = pipeline('question-answering')
context = """A novel coronavirus is a new coronavirus that has not been previously identified. The virus causing coronavirus disease 2019 (COVID-19), is not the same as the coronaviruses that commonly circulate among humans 
and cause mild illness, like the common cold. 
"""
result = nlp(context=context, question="what is a novel coronavirus?")

/Users/blessingmagabane/opt/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


In [3]:
result

{'score': 0.8791074156761169,
 'start': 23,
 'end': 81,
 'answer': 'a new coronavirus that has not been previously identified.'}

Under BERT, we use the bert-base-cased model. 

The tokenizer is shown below.

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

The pre-trained model is called.

In [5]:
model = TFBertForQuestionAnswering.from_pretrained('bert-base-cased', return_dict=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertForQuestionAnswering: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Defining the questions and context.

In [6]:
questions = [ 
    { "question": "what is a novel coronavirus?  ", "context": " A novel coronavirus is a new coronavirus that has not been previously identified. The virus causing coronavirus disease 2019 (COVID-19), is not the same as the coronaviruses that commonly circulate among humans and cause mild illness, like the common cold. " }, 
    { "question": "Why is the disease being called coronavirus disease 2019 COVID-19?", "context": " On February 11, 2020 the World Health Organization announced an official name for the disease that is causing the 2019 novel coronavirus outbreak, first identified in Wuhan China. The new name of this disease is coronavirus disease 2019, abbreviated as COVID-19. In COVID-19, CO stands for corona, VI for virus and D for disease. Formerly, this disease was referred to as 2019 novel coronavirus or 2019-nCoV " }, 
    { "question": "Will warm weather stop the outbreak of COVID-19?", "context": " It is not yet known whether weather and temperature affect the spread of COVID-19. Some other viruses, like those that cause the common cold and flu, spread more during cold weather months but that does not mean it is impossible to become sick with these viruses during other months.  There is much more to learn about the transmissibility, severity, and other features associated with COVID-19 and investigations are ongoing. " }, 
    { "question": "What is community spread?", "context": " Community spread means people have been infected with the virus in an area, including some who are not sure how or where they became infected. Each health department determines community spread differently based on local conditions. " }, 
    { "question": "Is it okay for me to donate blood?", "context": " In healthcare settings across the United States, donated blood is a lifesaving, essential part of caring for patients. The need for donated blood is constant, and blood centers are open and in urgent need of donations. CDC encourages people who are well to continue to donate blood if they are able, even if they are practicing social distancing because of COVID-19. CDC is supporting blood centers by providing recommendations that will keep donors and staff safe. Examples of these recommendations include spacing donor chairs 6 feet apart, thoroughly adhering to environmental cleaning practices, and encouraging donors to make donation appointments ahead of time. " }, 
    { "question": "What cleaning products should I use to protect against COVID-19?", "context": " Clean and disinfect frequently touched surfaces such as tables, doorknobs, light switches, countertops, handles, desks, phones, keyboards, toilets, faucets, and sinks.  If surfaces are dirty, clean them using detergent or soap and water prior to disinfection. To disinfect, most common EPA-registered household disinfectants will work. " }, 
    { "question": " What are the symptoms and complications that COVID-19 can cause?", "context": " People with COVID-19 have reported a wide range of symptoms – from mild symptoms to severe illness. Symptoms may appear 2-14 days after exposure to the virus. If you have fever, cough, or other symptoms, you might have COVID-19. " }, 
    { "question": "Can someone test negative and later test positive on a viral test for COVID-19?", "context": " Yes, it is possible. You may test negative if the sample was collected early in your infection and test positive later during this illness. You could also be exposed to COVID-19 after the test and get infected then. Even if you test negative, you still should take steps to protect yourself and others. " }, 
    { "question": "I recently around someone who has COVID-19 but I feel fine. Why should stay at home?", "context": " People with COVID-19 can still spread the virus even if they don’t have any symptoms. If you were around someone who had COVID-19, it is critical that you stay home and away from others for 14 days from the last day that you were around that person. Staying home and away from others at all times helps your health department in the fight against COVID-19 and helps protect you, your family, and your community. " } 
] 


Evaluating the questions and checking the answer. 

In [8]:
token_subset = 90
for row in questions:
  question, text = row["question"], row["context"]

In [9]:
input_dict = tokenizer(question, text, return_tensors='tf')
outputs = model(input_dict)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])
answer

'all times helps your health department in the fight against CO ##VI ##D - 19 and helps protect you , your family , and your'